In [1]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False   #ignore FutureWarning

import tensorflow as tf
tf.__version__

2024-03-19 17:17:45.509129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 17:17:45.849851: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-19 17:17:47.003755: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-19 17:17:47.003880: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

'2.11.0'

# Main Body

# 1.import librabies

In [2]:
import random
from tqdm import tqdm
import json

import pandas as pd
import numpy as np
import numpy.random as nr
from numpy.random import seed
import matplotlib.pyplot as plt
from glob import glob
from pathlib import Path
import cv2
import tensorflow.keras as keras
from tensorflow.keras import backend as K


from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split


from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.applications.vgg16 import preprocess_input
#from tensorflow.keras.applications.resnet50 import preprocess_input
#from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Dropout,MaxPooling2D, Activation, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import Callback

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
#clear memory in case of OOM
K.clear_session()

# 2. import data and random split

In [5]:
#set dictionary for disease and its index
disease_class = {'Atelectasis': 1,
                 'Cardiomegaly': 2,
                 'Effusion': 3,
                 'Infiltration': 4,
                 'Mass': 5,
                 'Nodule': 6,
                 'Pneumonia': 7,
                 'Pneumothorax': 8,
                 'Consolidation': 9,
                 'Edema': 10,
                 'Emphysema': 11,
                 'Fibrosis': 12,
                 'Pleural_Thickening': 13,
                 'Hernia': 14,
                 'No Finding': 15}

disease_rev = {v: k for k, v in disease_class.items()}

In [6]:
disease_img = {'Atelectasis': [],
                 'Cardiomegaly': [],
                 'Effusion': [],
                 'Infiltration': [],
                 'Mass': [],
                 'Nodule': [],
                 'Pneumonia': [],
                 'Pneumothorax': [],
                 'Consolidation': [],
                 'Edema': [],
                 'Emphysema': [],
                 'Fibrosis': [],
                 'Pleural_Thickening': [],
                 'Hernia': [],
                 'No Finding':[]}

In [7]:
#import labels of the images
data_ref = pd.read_csv("/media/ntu/volume1/home/s123md305_01/Documents/CXR8/Data_Entry_2017_v2020.csv")
pd.options.mode.chained_assignment = None        #ignore the SettingWithCopyWarning

#/media/ntu/volume1/home/s123md305_01/Documents/Generated/reconstructed_labels.csv
#/media/ntu/volume1/home/s123md305_01/Documents/CXR8/Data_Entry_2017_v2020.csv
for i in tqdm(range(len(data_ref))):
    #print(i)
    if "|" not in data_ref['Finding Labels'][i]:
        disease_img[data_ref['Finding Labels'][i]].append(data_ref['Image Index'][i])

100%|████████████████████████| 112120/112120 [00:02<00:00, 53705.15it/s]


In [8]:
simp_data_ref = data_ref[["Image Index", "Finding Labels"]]
simp_data_ref.set_index("Image Index", inplace = True)

# Training with Noise/GAN augmented images

In [9]:
number = 10000

img_names = []
for dis in disease_img.keys():
    num = round(number/91324*len(disease_img[dis]))
    for i in range(num):
        img_names.append(disease_img[dis][i])

In [10]:
train_image = []
y = np.zeros(shape = (len(img_names), len(disease_class.keys())))

for i in tqdm(range(len(img_names))):

    #img = image.load_img('Noise/N'+img_names[i],target_size=(112,112,3))
    img = image.load_img('/media/ntu/volume1/home/s123md305_01/Documents/Generated/ComGenerated224/'+img_names[i],target_size=(112,112,3))      #GG:cycleGAN; G:DCGAN
    img = image.img_to_array(img)
    train_image.append(img)
    
    for j in range(len(disease_class.keys())):
        if disease_rev[j+1] == simp_data_ref['Finding Labels'][img_names[i]]:
            y[i][j] = 1
            
X = np.array(train_image)

  0%|                                                 | 0/10000 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/media/ntu/volume1/home/s123md305_01/Documents/Generated/ComGenerated224/00000011_006.png'

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42, test_size = 0.15, stratify = y)

# 3. Data preprocessing

In [13]:
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)

In [41]:
#custom loss function
#https://www.programmersought.com/article/60001511310/
def focal_loss(alpha = 0.5, beta = 2.0):

    epsilon = 1.e-7
    def loss_fn2(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    
        alpha_t = y_true*alpha + (tf.ones_like(y_true)-y_true)*(1-alpha)
        y_t = tf.multiply(y_true, y_pred) + tf.multiply(1-y_true, 1-y_pred)
        ce = -tf.math.log(y_t)
        weight = tf.pow(tf.subtract(1., y_t), beta)
        fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
        loss = tf.reduce_mean(fl)
        return loss
    
    return loss_fn2

# 5.Train With Original images

In [8]:
number = 10000

img_names = []
for dis in disease_img.keys():
    num = round(number/91324*len(disease_img[dis]))
    for i in range(num):
        img_names.append(disease_img[dis][i])

In [11]:
X = []
train_image = []
y = np.zeros(shape = (len(img_names), len(disease_class.keys())))

for i in tqdm(range(len(img_names))):
    
    img = image.load_img('/media/ntu/volume1/home/s123md305_01/Documents/Dataset/'+img_names[i],target_size=(224,224,3))
    img = image.img_to_array(img)
    train_image.append(img)
    
    for j in range(len(disease_class.keys())):
        if disease_rev[j+1] == simp_data_ref['Finding Labels'][img_names[i]]:
            y[i][j] = 1
            
X = np.array(train_image)

100%|█████████████████████████████████████| 10000/10000 [02:55<00:00, 57.08it/s]


In [64]:
X_train, X_te, y_train, y_te = train_test_split(X, y, random_state = 42, test_size = 0.45, stratify = y)
X_test, X_val, y_test, y_val = train_test_split(X_te, y_te, random_state = 42, test_size = 0.33, stratify = y_te)


#0.45, 0.33 - when model has been trained with augmentation
#0.3, 0.5 - just original images

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [13]:
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)
y_train = preprocess_input(y_train)
y_test = preprocess_input(y_test)

# 6. Training Performance

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, EfficientNetB0
from tensorflow.keras.applications.vgg16 import preprocess_input
#from tensorflow.keras.applications.resnet50 import preprocess_input
#from tensorflow.keras.applications.densenet import preprocess_input
from sklearn.metrics import roc_auc_score
import numpy as np
# Assuming you have loaded and preprocessed your data
# X_train, X_test, y_train, y_test = your_data_loading_function()
# Make sure you have the correct input shape and num_classes defined based on your dataset
input_shape = (224, 224, 3)  # Example input shape, adjust based on your data
num_classes = 15  # Change this to the number of classes in your dataset

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.applications import VGG16
import tensorflow.keras.backend as K

# Define Focal Loss
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.sum((1-alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

def get_model(model_name, input_shape, num_classes):
    if model_name == 'EfficientNetB0':
        base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Unsupported model")

    # Add custom top layers
    x = GlobalAveragePooling2D()(base_model.output)
    output = Dense(num_classes, activation='softmax')(x)  # Adjust for number of classes and activation
    model = tf.keras.Model(inputs=base_model.input, outputs=output)

    # Compile the model with Adam optimizer and Focal Loss
    adam = Adam(learning_rate=0.0001)
    model.compile(optimizer=adam, loss=focal_loss(), metrics=['accuracy'])
    return model

# Train and evaluate each model
def compute_aauc_and_sdv(model, X_test, y_test, num_classes):
    # Ensure y_test is one-hot encoded. If it's not, you'll need to adjust this part.
    # Predict probabilities for each class
    y_pred = model.predict(X_test)
    
    auc_scores = []
    for i in range(num_classes):
        # Compute AUC for each class, treating it as a binary classification problem
        # This requires that your y_test is one-hot encoded
        auc = roc_auc_score(y_test[:, i], y_pred[:, i])
        auc_scores.append(auc)
    
    # Calculate AAUC (mean AUC) and SDV (standard deviation of AUCs)
    aauc = np.mean(auc_scores)
    sdv = np.std(auc_scores)
    return aauc, sdv

# Updated results dictionary to store AAUC and SDV along with accuracy
results = {}
for model_name in ['EfficientNetB0']:
    model = get_model(model_name, input_shape, num_classes)
    # Fit the model (assuming you have defined X_train, y_train, X_test, and y_test)
    history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)
    scores = model.evaluate(X_test, y_test)
    # Compute AAUC and SDV
    aauc, sdv = compute_aauc_and_sdv(model, X_test, y_test, num_classes)
    results[model_name] = {'Accuracy': scores[1], 'AAUC': aauc, 'SDV': sdv}

# Display the results
for model_name, metrics in results.items():
    print(f"{model_name}: Accuracy={metrics['Accuracy']:.4f}, AAUC={metrics['AAUC']:.4f}, SDV={metrics['SDV']:.4f}")


Epoch 1/10


2024-02-27 12:42:51.698273: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-02-27 12:43:19.726753: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 110.25MiB (rounded to 115605504)requested by op model_1/block2e_bn/FusedBatchNormV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-02-27 12:43:19.727350: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2024-02-27 12:43:19.727380: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 908, Chunks in use: 908. 227.0KiB allocated for chunks. 227.0KiB in use in bin. 87.9KiB clie

ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/block2e_bn/FusedBatchNormV3' defined at (most recent call last):
    File "/media/ntu/volume1/home/s123md305_01/anaconda3/envs/stylegan/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/media/ntu/volume1/home/s123md305_01/anaconda3/envs/stylegan/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/media/ntu/volume1/home/s123md305_01/anaconda3/envs/stylegan/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/media/ntu/volume1/home/s123md305_01/anaconda3/envs/stylegan/lib/python3.8/site-packages/traitlets/config/application.py", line 1053, in launch_instance
      app.start()
    File "/media/ntu/volume1/home/s123md305_01/anaconda3/envs/stylegan/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 737, in start
      self.io_loop.start()
    File

In [15]:
from sklearn.metrics import roc_auc_score
import numpy as np
from tensorflow.keras.utils import to_categorical

# Assuming num_classes is defined based on your dataset
# Assuming input_shape is defined based on your dataset (e.g., (112, 112, 3))
# Ensure X_test and y_test are prepared and correctly formatted for your model

def compute_aauc_and_sdv(model, X_test, y_test_one_hot):
    # Predict probabilities for each class
    y_pred = model.predict(X_test)
    
    auc_scores = []
    for i in range(num_classes):
        # Compute AUC for each class, treating it as a binary classification problem
        auc = roc_auc_score(y_test_one_hot[:, i], y_pred[:, i])
        auc_scores.append(auc)
    
    # Calculate AAUC (mean AUC) and SDV (standard deviation of AUCs)
    aauc = np.mean(auc_scores)
    sdv = np.std(auc_scores)
    return aauc, sdv

results = {}
for model_name in ['VGG16', 'ResNet50', 'DenseNet121']:
    model = get_model(model_name, input_shape, num_classes)  # Ensure this function is correctly defined as previously discussed
    
    # Convert y_test to one-hot encoding if not already done
    y_test_one_hot = to_categorical(y_test, num_classes=num_classes)
    
    # Compute AAUC and SDV
    aauc, sdv = compute_aauc_and_sdv(model, X_test, y_test_one_hot)
    results[model_name] = (aauc, sdv)

# Display the AAUC and SDV results
for model_name, (aauc, sdv) in results.items():
    print(f"{model_name}: AAUC={aauc:.4f}, SDV={sdv:.4f}")


571/571 [==============================] - 9s 15ms/step


ValueError: Expected 2D array, got 1D array instead:
array=[0.03699585 0.03400383 0.0309319  ... 0.0442787  0.03871683 0.03024635].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [24]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
#from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input
#from tensorflow.keras.applications.densenet import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping
input_shape = (112, 112, 3)  # Example input shape for a typical image dataset
num_classes = 15  # Change this to match the number of classes in your dataset


# Function to define and compile the model
def build_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=[tf.keras.metrics.AUC(name='auc')])
    return model

# Function for Focal Loss
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -tf.reduce_sum(alpha * tf.pow(1. - pt_1, gamma) * tf.math.log(pt_1)) - tf.reduce_sum((1-alpha) * tf.pow(pt_0, gamma) * tf.math.log(1. - pt_0))
    return focal_loss_fixed

# Number of runs to calculate the standard deviation
n_runs = 5
auc_scores = []

for run in range(n_runs):
    tf.keras.backend.clear_session()
    # Assuming X and y are your complete dataset excluding the test set
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42+run)

    # Preprocess the test set
    X_test = preprocess_input(X_test)
   
    # Split the training + validation set into actual training and validation sets (82.35:17.65)
    # This will give you 70% of the total data for training and 15% of the total data for validation
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42+run)

    # Preprocess the training and validation sets
    X_train = preprocess_input(X_train)
    X_val = preprocess_input(X_val)

    model = build_model(input_shape, num_classes)

    early_stopping = EarlyStopping(monitor='val_auc', patience=3, mode='max', verbose=1)
    
    model.fit(
        X_train, y_train,
        epochs=20,  # Adjust based on your dataset and model's performance
        batch_size=10,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1  # Set to 0 to reduce log messages
    )

    # Evaluate the model on your test set, assuming X_test, y_test are your test data and labels
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
    auc_scores.append(auc)
    print(f"Run {run+1}/{n_runs}, Test AUC: {auc:.4f}")

# Calculate and print the standard deviation of AUC scores
auc_std_dev = np.std(auc_scores)
print(f"Standard Deviation of AUC over {n_runs} runs: {auc_std_dev:.4f}")


Epoch 1/20
700/700 [==============================] - 22s 24ms/step - loss: 3.6492 - auc: 0.8736 - val_loss: 3.0480 - val_auc: 0.9023
Epoch 2/20
700/700 [==============================] - 12s 17ms/step - loss: 3.0700 - auc: 0.9020 - val_loss: 2.9399 - val_auc: 0.9052
Epoch 3/20
700/700 [==============================] - 12s 18ms/step - loss: 2.8640 - auc: 0.9130 - val_loss: 2.8560 - val_auc: 0.9108
Epoch 4/20
700/700 [==============================] - 12s 18ms/step - loss: 2.7051 - auc: 0.9222 - val_loss: 2.8426 - val_auc: 0.9123
Epoch 5/20
700/700 [==============================] - 12s 18ms/step - loss: 2.5957 - auc: 0.9284 - val_loss: 2.7998 - val_auc: 0.9146
Epoch 6/20
700/700 [==============================] - 12s 18ms/step - loss: 2.4974 - auc: 0.9338 - val_loss: 2.8559 - val_auc: 0.9105
Epoch 7/20
700/700 [==============================] - 12s 18ms/step - loss: 2.4270 - auc: 0.9374 - val_loss: 2.7762 - val_auc: 0.9165
Epoch 8/20
700/700 [==============================] - 12s 18ms

In [14]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
from tensorflow.keras.applications.vgg16 import preprocess_input
#from tensorflow.keras.applications.resnet50 import preprocess_input
#from tensorflow.keras.applications.densenet import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping
input_shape = (224, 224, 3)  # Example input shape for a typical image dataset
num_classes = 15  # Change this to match the number of classes in your dataset


# Function to define and compile the model
def build_model(input_shape, num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=[tf.keras.metrics.AUC(name='auc')])
    return model

# Function for Focal Loss
#https://www.programmersought.com/article/60001511310/
def focal_loss(alpha = 0.5, beta = 2.0):
    epsilon = 1.e-7
    def loss_fn2(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    
        alpha_t = y_true*alpha + (tf.ones_like(y_true)-y_true)*(1-alpha)
        y_t = tf.multiply(y_true, y_pred) + tf.multiply(1-y_true, 1-y_pred)
        ce = -tf.math.log(y_t)
        weight = tf.pow(tf.subtract(1., y_t), beta)
        fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
        loss = tf.reduce_mean(fl)
        return loss
    
    return loss_fn2

# Number of runs to calculate the standard deviation
n_runs = 5
auc_scores = []

for run in range(n_runs):
    tf.keras.backend.clear_session()
    # Assuming X and y are your complete dataset excluding the test set
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42+run)

    # Preprocess the test set
    X_test = preprocess_input(X_test)
   
    # Split the training + validation set into actual training and validation sets (82.35:17.65)
    # This will give you 70% of the total data for training and 15% of the total data for validation
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42+run)

    # Preprocess the training and validation sets
    X_train = preprocess_input(X_train)
    X_val = preprocess_input(X_val)

    model = build_model(input_shape, num_classes)

    early_stopping = EarlyStopping(monitor='val_auc', patience=3, mode='max', verbose=1)
    
    model.fit(
        X_train, y_train,
        epochs=20,  # Adjust based on your dataset and model's performance
        batch_size=10,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1  # Set to 0 to reduce log messages
    )

    # Evaluate the model on your test set, assuming X_test, y_test are your test data and labels
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
    auc_scores.append(auc)
    print(f"Run {run+1}/{n_runs}, Test AUC: {auc:.4f}")

# Calculate and print the standard deviation of AUC scores
auc_std_dev = np.std(auc_scores)
print(f"Standard Deviation of AUC over {n_runs} runs: {auc_std_dev:.4f}")


Epoch 1/20
700/700 [==============================] - 29s 38ms/step - loss: 0.0931 - auc: 0.7912 - val_loss: 0.0680 - val_auc: 0.8503
Epoch 2/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0684 - auc: 0.8411 - val_loss: 0.0593 - val_auc: 0.8720
Epoch 3/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0598 - auc: 0.8565 - val_loss: 0.0540 - val_auc: 0.8750
Epoch 4/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0543 - auc: 0.8672 - val_loss: 0.0500 - val_auc: 0.8806
Epoch 5/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0503 - auc: 0.8764 - val_loss: 0.0473 - val_auc: 0.8873
Epoch 6/20
700/700 [==============================] - 23s 32ms/step - loss: 0.0473 - auc: 0.8842 - val_loss: 0.0462 - val_auc: 0.8910
Epoch 7/20
700/700 [==============================] - 23s 32ms/step - loss: 0.0448 - auc: 0.8908 - val_loss: 0.0440 - val_auc: 0.8930
Epoch 8/20
700/700 [==============================] - 23s 32ms

2024-03-10 20:51:50.769134: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 884.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-10 20:51:50.808469: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 884.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-10 20:51:50.882416: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 884.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-10 20:51:50.948723: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocato

 8/47 [====>.........................] - ETA: 2s

2024-03-10 20:51:51.011620: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 884.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-10 20:51:51.076400: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 884.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-10 20:51:51.144860: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 884.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-10 20:51:51.207858: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocato

12/47 [======>.......................] - ETA: 2s

2024-03-10 20:51:51.271289: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 884.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-03-10 20:51:51.335200: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 884.25MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


47/47 [==============================] - 3s 67ms/step
Run 3/5, Test AUC: 0.6663
Epoch 1/20
700/700 [==============================] - 26s 35ms/step - loss: 0.0947 - auc: 0.7941 - val_loss: 0.0721 - val_auc: 0.8381
Epoch 2/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0658 - auc: 0.8506 - val_loss: 0.0639 - val_auc: 0.8539
Epoch 3/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0582 - auc: 0.8646 - val_loss: 0.0594 - val_auc: 0.8549
Epoch 4/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0535 - auc: 0.8740 - val_loss: 0.0562 - val_auc: 0.8711
Epoch 5/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0498 - auc: 0.8820 - val_loss: 0.0534 - val_auc: 0.8713
Epoch 6/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0471 - auc: 0.8879 - val_loss: 0.0507 - val_auc: 0.8755
Epoch 7/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0447 - auc: 0.8936 - val_loss: 0.0493 

In [12]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
#from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input
#from tensorflow.keras.applications.densenet import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping
input_shape = (224, 224, 3)  # Example input shape for a typical image dataset
num_classes = 15  # Change this to match the number of classes in your dataset


# Function to define and compile the model
def build_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=[tf.keras.metrics.AUC(name='auc')])
    return model

# Function for Focal Loss
#https://www.programmersought.com/article/60001511310/
def focal_loss(alpha = 0.5, beta = 2.0):
    epsilon = 1.e-7
    def loss_fn2(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    
        alpha_t = y_true*alpha + (tf.ones_like(y_true)-y_true)*(1-alpha)
        y_t = tf.multiply(y_true, y_pred) + tf.multiply(1-y_true, 1-y_pred)
        ce = -tf.math.log(y_t)
        weight = tf.pow(tf.subtract(1., y_t), beta)
        fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
        loss = tf.reduce_mean(fl)
        return loss
    
    return loss_fn2

# Number of runs to calculate the standard deviation
n_runs = 5
auc_scores = []

for run in range(n_runs):
    tf.keras.backend.clear_session()
    # Assuming X and y are your complete dataset excluding the test set
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42+run)

    # Preprocess the test set
    X_test = preprocess_input(X_test)
   
    # Split the training + validation set into actual training and validation sets (82.35:17.65)
    # This will give you 70% of the total data for training and 15% of the total data for validation
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42+run)

    # Preprocess the training and validation sets
    X_train = preprocess_input(X_train)
    X_val = preprocess_input(X_val)

    model = build_model(input_shape, num_classes)

    early_stopping = EarlyStopping(monitor='val_auc', patience=3, mode='max', verbose=1)
    
    model.fit(
        X_train, y_train,
        epochs=20,  # Adjust based on your dataset and model's performance
        batch_size=10,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1  # Set to 0 to reduce log messages
    )

    # Evaluate the model on your test set, assuming X_test, y_test are your test data and labels
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
    auc_scores.append(auc)
    print(f"Run {run+1}/{n_runs}, Test AUC: {auc:.4f}")

# Calculate and print the standard deviation of AUC scores
auc_std_dev = np.std(auc_scores)
print(f"Standard Deviation of AUC over {n_runs} runs: {auc_std_dev:.4f}")


2024-03-10 19:31:10.673250: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 19:31:15.262703: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:19:00.0, compute capability: 8.6
2024-03-10 19:31:15.264508: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22288 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2024-03-10 19:31:15.266261: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localho

Epoch 1/20


2024-03-10 19:31:40.602781: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101
2024-03-10 19:31:42.364200: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-10 19:31:44.148029: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-03-10 19:31:44.152892: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x557046f4db00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-10 19:31:44.152930: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-10 19:31:44.152937: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-10 19:31:44.152942: I tensorflow/compiler/xla/s

700/700 [==============================] - 31s 29ms/step - loss: 0.0404 - auc: 0.8856 - val_loss: 0.0340 - val_auc: 0.9123
Epoch 2/20
700/700 [==============================] - 15s 22ms/step - loss: 0.0340 - auc: 0.9142 - val_loss: 0.0331 - val_auc: 0.9177
Epoch 3/20
700/700 [==============================] - 15s 22ms/step - loss: 0.0323 - auc: 0.9224 - val_loss: 0.0323 - val_auc: 0.9204
Epoch 4/20
700/700 [==============================] - 15s 22ms/step - loss: 0.0313 - auc: 0.9270 - val_loss: 0.0322 - val_auc: 0.9214
Epoch 5/20
700/700 [==============================] - 15s 22ms/step - loss: 0.0303 - auc: 0.9320 - val_loss: 0.0322 - val_auc: 0.9254
Epoch 6/20
700/700 [==============================] - 15s 22ms/step - loss: 0.0294 - auc: 0.9356 - val_loss: 0.0323 - val_auc: 0.9220
Epoch 7/20
700/700 [==============================] - 15s 22ms/step - loss: 0.0288 - auc: 0.9381 - val_loss: 0.0311 - val_auc: 0.9267
Epoch 8/20
700/700 [==============================] - 15s 22ms/step - los

Epoch 15/20
700/700 [==============================] - 16s 22ms/step - loss: 0.0248 - auc: 0.9543 - val_loss: 0.0301 - val_auc: 0.9315
Epoch 16/20
700/700 [==============================] - 16s 22ms/step - loss: 0.0244 - auc: 0.9554 - val_loss: 0.0303 - val_auc: 0.9313
Epoch 17/20
700/700 [==============================] - 16s 22ms/step - loss: 0.0239 - auc: 0.9571 - val_loss: 0.0300 - val_auc: 0.9326
Epoch 18/20
700/700 [==============================] - 16s 22ms/step - loss: 0.0236 - auc: 0.9580 - val_loss: 0.0310 - val_auc: 0.9281
Epoch 19/20
700/700 [==============================] - 16s 23ms/step - loss: 0.0233 - auc: 0.9594 - val_loss: 0.0302 - val_auc: 0.9316
Epoch 20/20
700/700 [==============================] - 16s 22ms/step - loss: 0.0230 - auc: 0.9602 - val_loss: 0.0304 - val_auc: 0.9304
Epoch 20: early stopping
47/47 [==============================] - 4s 40ms/step
Run 5/5, Test AUC: 0.7109
Standard Deviation of AUC over 5 runs: 0.0021


In [13]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
#from tensorflow.keras.applications.vgg16 import preprocess_input
#from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping
input_shape = (224, 224, 3)  # Example input shape for a typical image dataset
num_classes = 15  # Change this to match the number of classes in your dataset


# Function to define and compile the model
def build_model(input_shape, num_classes):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=[tf.keras.metrics.AUC(name='auc')])
    return model

# Function for Focal Loss
#https://www.programmersought.com/article/60001511310/
def focal_loss(alpha = 0.5, beta = 2.0):
    epsilon = 1.e-7
    def loss_fn2(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    
        alpha_t = y_true*alpha + (tf.ones_like(y_true)-y_true)*(1-alpha)
        y_t = tf.multiply(y_true, y_pred) + tf.multiply(1-y_true, 1-y_pred)
        ce = -tf.math.log(y_t)
        weight = tf.pow(tf.subtract(1., y_t), beta)
        fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
        loss = tf.reduce_mean(fl)
        return loss
    
    return loss_fn2

# Number of runs to calculate the standard deviation
n_runs = 5
auc_scores = []

for run in range(n_runs):
    tf.keras.backend.clear_session()
    # Assuming X and y are your complete dataset excluding the test set
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42+run)

    # Preprocess the test set
    X_test = preprocess_input(X_test)
   
    # Split the training + validation set into actual training and validation sets (82.35:17.65)
    # This will give you 70% of the total data for training and 15% of the total data for validation
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42+run)

    # Preprocess the training and validation sets
    X_train = preprocess_input(X_train)
    X_val = preprocess_input(X_val)

    model = build_model(input_shape, num_classes)

    early_stopping = EarlyStopping(monitor='val_auc', patience=3, mode='max', verbose=1)
    
    model.fit(
        X_train, y_train,
        epochs=20,  # Adjust based on your dataset and model's performance
        batch_size=10,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1  # Set to 0 to reduce log messages
    )

    # Evaluate the model on your test set, assuming X_test, y_test are your test data and labels
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
    auc_scores.append(auc)
    print(f"Run {run+1}/{n_runs}, Test AUC: {auc:.4f}")

# Calculate and print the standard deviation of AUC scores
auc_std_dev = np.std(auc_scores)
print(f"Standard Deviation of AUC over {n_runs} runs: {auc_std_dev:.4f}")


Epoch 1/20
700/700 [==============================] - 38s 38ms/step - loss: 0.0428 - auc: 0.8726 - val_loss: 0.0356 - val_auc: 0.9074
Epoch 2/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0365 - auc: 0.9012 - val_loss: 0.0342 - val_auc: 0.9123
Epoch 3/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0352 - auc: 0.9072 - val_loss: 0.0335 - val_auc: 0.9155
Epoch 4/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0343 - auc: 0.9120 - val_loss: 0.0327 - val_auc: 0.9188
Epoch 5/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0335 - auc: 0.9157 - val_loss: 0.0322 - val_auc: 0.9212
Epoch 6/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0329 - auc: 0.9186 - val_loss: 0.0320 - val_auc: 0.9215
Epoch 7/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0323 - auc: 0.9217 - val_loss: 0.0322 - val_auc: 0.9211
Epoch 8/20
700/700 [==============================] - 22s 31ms

Epoch 10/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0304 - auc: 0.9309 - val_loss: 0.0344 - val_auc: 0.9133
Epoch 11/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0301 - auc: 0.9324 - val_loss: 0.0341 - val_auc: 0.9151
Epoch 12/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0298 - auc: 0.9338 - val_loss: 0.0342 - val_auc: 0.9142
Epoch 13/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0296 - auc: 0.9348 - val_loss: 0.0340 - val_auc: 0.9149
Epoch 14/20
700/700 [==============================] - 22s 31ms/step - loss: 0.0292 - auc: 0.9363 - val_loss: 0.0340 - val_auc: 0.9150
Epoch 14: early stopping
47/47 [==============================] - 5s 48ms/step
Run 4/5, Test AUC: 0.6980
Epoch 1/20
700/700 [==============================] - 36s 37ms/step - loss: 0.0392 - auc: 0.8894 - val_loss: 0.0352 - val_auc: 0.9075
Epoch 2/20
700/700 [==============================] - 22s 32ms/step - loss: 0.0355 - a

In [ ]:
number = 10000

img_names = []
for dis in disease_img.keys():
    num = round(number/91324*len(disease_img[dis]))
    for i in range(num):
        img_names.append(disease_img[dis][i])
        
X = []
train_image = []
y = np.zeros(shape = (len(img_names), len(disease_class.keys())))

for i in tqdm(range(len(img_names))):
    
    img = image.load_img('/media/ntu/volume1/home/s123md305_01/Documents/Dataset/'+img_names[i],target_size=(224,224,3))
    img = image.img_to_array(img)
    train_image.append(img)
    
    for j in range(len(disease_class.keys())):
        if disease_rev[j+1] == simp_data_ref['Finding Labels'][img_names[i]]:
            y[i][j] = 1
            
X = np.array(train_image)

In [ ]:
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import numpy as np

# Assuming disease_img, disease_class, and simp_data_ref are predefined

for number in range(30000, 31000, 10000): 
    print(number)# From 10,000 to 90,000
    img_names = []
    for dis in disease_img.keys():
        num = round(number / 91324 * len(disease_img[dis]))
        for i in range(num):
            img_names.append(disease_img[dis][i])
            
    X = []
    train_image = []
    y = np.zeros(shape=(len(img_names), len(disease_class.keys())))

    for i in tqdm(range(len(img_names))):
        img = image.load_img('/media/ntu/volume1/home/s123md305_01/Documents/CombinedResized/Resized224/' + img_names[i], target_size=(224, 224, 3))
        img = image.img_to_array(img)
        train_image.append(img)
        
        for j in range(len(disease_class.keys())):
            if disease_rev[j + 1] == simp_data_ref['Finding Labels'][img_names[i]]:
                y[i][j] = 1
                
    X = np.array(train_image)
    
    import numpy as np
    import os
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
    #from tensorflow.keras.applications.vgg16 import preprocess_input
    from tensorflow.keras.applications.resnet50 import preprocess_input
    #from tensorflow.keras.applications.densenet import preprocess_input
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from tensorflow.keras.callbacks import EarlyStopping
    input_shape = (224, 224, 3)  # Example input shape for a typical image dataset
    num_classes = 15  # Change this to match the number of classes in your dataset


    # Function to define and compile the model
    def build_model(input_shape, num_classes):
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        for layer in base_model.layers:
            layer.trainable = False

        model = Sequential([
            base_model,
            GlobalAveragePooling2D(),
            Dense(num_classes, activation='softmax')
        ])

        model.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=[tf.keras.metrics.AUC(name='auc')])
        return model

    # Function for Focal Loss
    #https://www.programmersought.com/article/60001511310/
    def focal_loss(alpha = 0.5, beta = 2.0):
        epsilon = 1.e-7
        def loss_fn2(y_true, y_pred):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

            alpha_t = y_true*alpha + (tf.ones_like(y_true)-y_true)*(1-alpha)
            y_t = tf.multiply(y_true, y_pred) + tf.multiply(1-y_true, 1-y_pred)
            ce = -tf.math.log(y_t)
            weight = tf.pow(tf.subtract(1., y_t), beta)
            fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
            loss = tf.reduce_mean(fl)
            return loss

        return loss_fn2

    # Number of runs to calculate the standard deviation
    n_runs = 3
    auc_scores = []

    for run in range(n_runs):
        tf.keras.backend.clear_session()
        # Assuming X and y are your complete dataset excluding the test set
        X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42+run)

        # Preprocess the test set
        X_test = preprocess_input(X_test)

        # Split the training + validation set into actual training and validation sets (82.35:17.65)
        # This will give you 70% of the total data for training and 15% of the total data for validation
        X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42+run)

        # Preprocess the training and validation sets
        X_train = preprocess_input(X_train)
        X_val = preprocess_input(X_val)

        model = build_model(input_shape, num_classes)

        early_stopping = EarlyStopping(monitor='val_auc', patience=3, mode='max', verbose=1)

        model.fit(
            X_train, y_train,
            epochs=20,  # Adjust based on your dataset and model's performance
            batch_size=10,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=1  # Set to 0 to reduce log messages
        )

        # Evaluate the model on your test set, assuming X_test, y_test are your test data and labels
        y_pred = model.predict(X_test)
        auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
        auc_scores.append(auc)
        print(f"Run {run+1}/{n_runs}, Test AUC: {auc:.4f}")

    # Calculate and print the standard deviation of AUC scores
    auc_std_dev = np.std(auc_scores)
    aauc=np.mean(auc_scores)
    print(f"Standard Deviation of AUC over {n_runs} runs: {auc_std_dev:.4f}")
    print("aauc=",aauc)

        # Now, X and y contain the images and labels for this iteration
        # You can now proceed with training or saving this data


30000


100%|████████████████████████████████████| 30001/30001 [01:18<00:00, 382.97it/s]
2024-03-11 17:07:23.864827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 17:07:28.332516: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:19:00.0, compute capability: 8.6
2024-03-11 17:07:28.334326: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22288 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2024-03-11 17:07:28.336140: I t

Epoch 1/20


2024-03-11 17:08:26.769848: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101
2024-03-11 17:08:28.552131: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-11 17:08:30.319915: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-03-11 17:08:30.324863: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x5624417064c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-11 17:08:30.324902: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-11 17:08:30.324908: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-11 17:08:30.324914: I tensorflow/compiler/xla/s

2100/2100 [==============================] - 69s 28ms/step - loss: 0.0356 - auc: 0.9067 - val_loss: 0.0330 - val_auc: 0.9193
Epoch 2/20
2100/2100 [==============================] - 47s 22ms/step - loss: 0.0321 - auc: 0.9228 - val_loss: 0.0319 - val_auc: 0.9240
Epoch 3/20
2100/2100 [==============================] - 47s 22ms/step - loss: 0.0308 - auc: 0.9286 - val_loss: 0.0318 - val_auc: 0.9236
Epoch 4/20
2100/2100 [==============================] - 47s 22ms/step - loss: 0.0299 - auc: 0.9325 - val_loss: 0.0311 - val_auc: 0.9272
Epoch 5/20
2100/2100 [==============================] - 47s 22ms/step - loss: 0.0293 - auc: 0.9354 - val_loss: 0.0311 - val_auc: 0.9271
Epoch 6/20
2100/2100 [==============================] - 47s 22ms/step - loss: 0.0287 - auc: 0.9379 - val_loss: 0.0314 - val_auc: 0.9297
Epoch 7/20
2100/2100 [==============================] - 47s 22ms/step - loss: 0.0282 - auc: 0.9399 - val_loss: 0.0312 - val_auc: 0.9270
Epoch 8/20
2100/2100 [==============================] - 47s

In [9]:
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import numpy as np

# Assuming disease_img, disease_class, and simp_data_ref are predefined

for number in range(91324, 92324, 10000): 
    print(number)# From 10,000 to 90,000
    img_names = []
    for dis in disease_img.keys():
        num = round(number / 91324 * len(disease_img[dis]))
        for i in range(num):
            img_names.append(disease_img[dis][i])
            
    X = []
    train_image = []
    y = np.zeros(shape=(len(img_names), len(disease_class.keys())))

    for i in tqdm(range(len(img_names))):
        img = image.load_img('/media/ntu/volume1/home/s123md305_01/Documents/CombinedResized/Resized112/' + img_names[i], target_size=(112, 112, 3))
        img = image.img_to_array(img)
        train_image.append(img)
        
        for j in range(len(disease_class.keys())):
            if disease_rev[j + 1] == simp_data_ref['Finding Labels'][img_names[i]]:
                y[i][j] = 1
                
    X = np.array(train_image)
    
    import numpy as np
    import os
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
    #from tensorflow.keras.applications.vgg16 import preprocess_input
    from tensorflow.keras.applications.resnet50 import preprocess_input
    #from tensorflow.keras.applications.densenet import preprocess_input
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from tensorflow.keras.callbacks import EarlyStopping
    input_shape = (112, 112, 3)  # Example input shape for a typical image dataset
    num_classes = 15  # Change this to match the number of classes in your dataset


    # Function to define and compile the model
    def build_model(input_shape, num_classes):
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        for layer in base_model.layers:
            layer.trainable = False

        model = Sequential([
            base_model,
            GlobalAveragePooling2D(),
            Dense(num_classes, activation='softmax')
        ])

        model.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=[tf.keras.metrics.AUC(name='auc')])
        return model

    # Function for Focal Loss
    #https://www.programmersought.com/article/60001511310/
    def focal_loss(alpha = 0.5, beta = 2.0):
        epsilon = 1.e-7
        def loss_fn2(y_true, y_pred):
            y_true = tf.cast(y_true, tf.float32)
            y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

            alpha_t = y_true*alpha + (tf.ones_like(y_true)-y_true)*(1-alpha)
            y_t = tf.multiply(y_true, y_pred) + tf.multiply(1-y_true, 1-y_pred)
            ce = -tf.math.log(y_t)
            weight = tf.pow(tf.subtract(1., y_t), beta)
            fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
            loss = tf.reduce_mean(fl)
            return loss

        return loss_fn2

    # Number of runs to calculate the standard deviation
    n_runs = 1
    auc_scores = []

    for run in range(n_runs):
        tf.keras.backend.clear_session()
        # Assuming X and y are your complete dataset excluding the test set
        X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42+run)

        # Preprocess the test set
        X_test = preprocess_input(X_test)

        # Split the training + validation set into actual training and validation sets (82.35:17.65)
        # This will give you 70% of the total data for training and 15% of the total data for validation
        X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42+run)

        # Preprocess the training and validation sets
        X_train = preprocess_input(X_train)
        X_val = preprocess_input(X_val)

        model = build_model(input_shape, num_classes)

        early_stopping = EarlyStopping(monitor='val_auc', patience=3, mode='max', verbose=1)

        model.fit(
            X_train, y_train,
            epochs=20,  # Adjust based on your dataset and model's performance
            batch_size=10,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=1  # Set to 0 to reduce log messages
        )

        # Evaluate the model on your test set, assuming X_test, y_test are your test data and labels
        y_pred = model.predict(X_test)
        auc = roc_auc_score(y_test, y_pred, multi_class='ovo')
        auc_scores.append(auc)
        print(f"Run {run+1}/{n_runs}, Test AUC: {auc:.4f}")

    # Calculate and print the standard deviation of AUC scores
    auc_std_dev = np.std(auc_scores)
    print(f"Standard Deviation of AUC over {n_runs} runs: {auc_std_dev:.4f}")

        # Now, X and y contain the images and labels for this iteration
        # You can now proceed with training or saving this data


91324


100%|███████████████████████████████████| 91324/91324 [01:26<00:00, 1060.56it/s]
2024-03-11 15:19:18.674747: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 15:19:23.036140: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:19:00.0, compute capability: 8.6
2024-03-11 15:19:23.037948: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22288 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2024-03-11 15:19:23.039726: I t

Epoch 1/20


2024-03-11 15:20:07.453157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101
2024-03-11 15:20:09.232888: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-11 15:20:10.991902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-03-11 15:20:10.998759: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fd52802baa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-11 15:20:10.998809: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-11 15:20:10.998816: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-11 15:20:10.998821: I tensorflow/compiler/xla/s

6393/6393 [==============================] - 127s 18ms/step - loss: 0.0366 - auc: 0.9067 - val_loss: 0.0331 - val_auc: 0.9197
Epoch 2/20
6393/6393 [==============================] - 107s 17ms/step - loss: 0.0328 - auc: 0.9210 - val_loss: 0.0325 - val_auc: 0.9220
Epoch 3/20
6393/6393 [==============================] - 107s 17ms/step - loss: 0.0317 - auc: 0.9259 - val_loss: 0.0326 - val_auc: 0.9215
Epoch 4/20
6393/6393 [==============================] - 107s 17ms/step - loss: 0.0309 - auc: 0.9291 - val_loss: 0.0326 - val_auc: 0.9236
Epoch 5/20
6393/6393 [==============================] - 108s 17ms/step - loss: 0.0303 - auc: 0.9317 - val_loss: 0.0320 - val_auc: 0.9230
Epoch 6/20
6393/6393 [==============================] - 109s 17ms/step - loss: 0.0299 - auc: 0.9335 - val_loss: 0.0319 - val_auc: 0.9238
Epoch 7/20
6393/6393 [==============================] - 108s 17ms/step - loss: 0.0295 - auc: 0.9352 - val_loss: 0.0325 - val_auc: 0.9215
Epoch 8/20
6393/6393 [==============================

In [10]:
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import numpy as np

# Assuming disease_img, disease_class, and simp_data_ref are predefined
for number in range(10000, 11000, 10000): 
    print(number)# From 10,000 to 90,000
    img_names = []
    for dis in disease_img.keys():
        num = round(number / 91324 * len(disease_img[dis]))
        for i in range(num):
            img_names.append(disease_img[dis][i])
            
    X = []
    train_image = []
    y = np.zeros(shape=(len(img_names), len(disease_class.keys())))

    for i in tqdm(range(len(img_names))):
        img = image.load_img('/media/ntu/volume1/home/s123md305_01/Documents/CombinedResized/Resized224/' + img_names[i], target_size=(224, 224, 3))
        img = image.img_to_array(img)
        train_image.append(img)
        
        for j in range(len(disease_class.keys())):
            if disease_rev[j + 1] == simp_data_ref['Finding Labels'][img_names[i]]:
                y[i][j] = 1
                
    X = np.array(train_image)

10000


100%|████████████████████████████| 10000/10000 [01:32<00:00, 108.55it/s]


In [17]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
from tensorflow.keras.applications.vgg16 import preprocess_input
#from tensorflow.keras.applications.resnet50 import preprocess_input
#from tensorflow.keras.applications.densenet import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping
input_shape = (224, 224, 3)  # Example input shape for a typical image dataset
num_classes = 15  # Change this to match the number of classes in your dataset

    # Function to define and compile the model
def build_model(input_shape, num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(num_classes, activation='softmax', kernel_initializer='glorot_uniform', dtype='float32')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=[tf.keras.metrics.AUC(name='auc')])
    return model

# Function for Focal Loss
#https://www.programmersought.com/article/60001511310/
def focal_loss(alpha = 0.5, beta = 2.0):
    epsilon = 1.e-7
    def loss_fn2(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        alpha_t = y_true*alpha + (tf.ones_like(y_true)-y_true)*(1-alpha)
        y_t = tf.multiply(y_true, y_pred) + tf.multiply(1-y_true, 1-y_pred)
        ce = -tf.math.log(y_t)
        weight = tf.pow(tf.subtract(1., y_t), beta)
        fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
        loss = tf.reduce_mean(fl)
        return loss

    return loss_fn2

# Number of runs to calculate the standard deviation
n_runs = 3
auc_scores = []

for run in range(n_runs):
    tf.keras.backend.clear_session()
    # Assuming X and y are your complete dataset excluding the test set
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42+run)

    # Preprocess the test set
    X_test = preprocess_input(X_test)

    # Split the training + validation set into actual training and validation sets (82.35:17.65)
    # This will give you 70% of the total data for training and 15% of the total data for validation
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42+run)

    # Preprocess the training and validation sets
    X_train = preprocess_input(X_train)
    X_val = preprocess_input(X_val)

    model = build_model(input_shape, num_classes)

    early_stopping = EarlyStopping(monitor='val_auc', patience=3, verbose=1)

    model.fit(
        X_train, y_train,
        epochs=20,  # Adjust based on your dataset and model's performance
        batch_size=10,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1  # Set to 0 to reduce log messages
    )

    # Evaluate the model on your test set, assuming X_test, y_test are your test data and labels
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, average = 'macro')
    auc_scores.append(auc)
    print(f"Run {run+1}/{n_runs}, Test AUC: {auc:.4f}")

# Calculate and print the standard deviation of AUC scores
auc_std_dev = np.std(auc_scores)
aauc=np.mean(auc_scores)
print(f"Standard Deviation of AUC over {n_runs} runs: {auc_std_dev:.4f}")
print("aauc=",aauc)

    # Now, X and y contain the images and labels for this iteration
    # You can now proceed with training or saving this data


Epoch 1/20
700/700 [==============================] - 31s 42ms/step - loss: 0.0912 - auc: 0.8048 - val_loss: 0.0654 - val_auc: 0.8562
Epoch 2/20
700/700 [==============================] - 27s 38ms/step - loss: 0.0645 - auc: 0.8551 - val_loss: 0.0583 - val_auc: 0.8646
Epoch 3/20
700/700 [==============================] - 27s 38ms/step - loss: 0.0573 - auc: 0.8656 - val_loss: 0.0539 - val_auc: 0.8744
Epoch 4/20
700/700 [==============================] - 27s 39ms/step - loss: 0.0524 - auc: 0.8757 - val_loss: 0.0501 - val_auc: 0.8810
Epoch 5/20
700/700 [==============================] - 27s 39ms/step - loss: 0.0487 - auc: 0.8827 - val_loss: 0.0484 - val_auc: 0.8865
Epoch 6/20
700/700 [==============================] - 27s 39ms/step - loss: 0.0459 - auc: 0.8898 - val_loss: 0.0461 - val_auc: 0.8907
Epoch 7/20
700/700 [==============================] - 27s 39ms/step - loss: 0.0437 - auc: 0.8953 - val_loss: 0.0448 - val_auc: 0.8934
Epoch 8/20
700/700 [==============================] - 27s 39ms

47/47 [==============================] - 4s 79ms/step
Run 3/3, Test AUC: 0.6528
Standard Deviation of AUC over 3 runs: 0.0050
aauc= 0.6594708127982392


In [13]:
import gc

In [16]:
gc.collect()

6636